### Table of Contents

1. Loading dataframes
2. Task 1
    - 2.1 Using sqldf (for reference)
    - 2.2 Using standard functions
        - Total comments per user
        - Total questions and answers per user
        - Score per user
        - Accepted answers per user
        - Final dataframe
    - 2.3 Using dplyr package
    - 2.4 Comparison and thoughts
3. Task 2
    - 3.1 Using sqldf (for reference)
    - 3.2 Using standard R functions
        - Users with relevant badges
        - Posts by location
        - Badges by location
        - Final dataframe
    - 3.3 Using dplyr package
    - 3.4 Comparison and thoughts
4. Task 3
    - 4.1 Using sqldf (for reference)
    - 4.2 Using standard R functions
    - 4.3 Explanation of the query



# 1. Loading dataframes and libraries

In [18]:
Badges <- read.csv("data/badges.csv")
Comments <- read.csv("data/comments.csv")
PostLinks <- read.csv("data/postLinks.csv")
Posts <- read.csv("data/posts.csv")
Tags <- read.csv("data/tags.csv")
Users <- read.csv("data/users.csv")
Votes <- read.csv("data/votes.csv")


In [19]:
print(nrow(Badges))
print(nrow(Comments))
print(nrow(PostLinks))
print(nrow(Posts))
print(nrow(Tags))
print(nrow(Users))
print(nrow(Votes))


[1] 106248
[1] 164608
[1] 6637
[1] 70619
[1] 579
[1] 56104
[1] 354103


In [20]:
library(dplyr)
library(microbenchmark)
library(sqldf)


# 2.Task 1

## 2.1 Using sqldf (for reference)

In [21]:
tmp <- sqldf("
  SELECT 
    u.Id as UserId,
    u.DisplayName,
    COUNT(DISTINCT CASE WHEN p.PostTypeId = 1 THEN p.Id END) as TotalQuestions,
    COUNT(DISTINCT CASE WHEN p.PostTypeId = 2 THEN p.Id END) as TotalAnswers,
    COUNT(DISTINCT c.Id) as TotalComments,
    SUM(p.Score) as TotalScore,
    COUNT(
      DISTINCT CASE WHEN p2.AcceptedAnswerId = p.Id THEN p.Id END
    ) as AcceptedAnswers
  FROM Users u
  LEFT JOIN Posts p ON u.Id = p.OwnerUserId
  LEFT JOIN Comments c ON u.Id = c.UserId
  LEFT JOIN Posts p2 ON p2.AcceptedAnswerId = p.Id
  GROUP BY u.Id, u.DisplayName
  HAVING AcceptedAnswers > 0
  ORDER BY AcceptedAnswers DESC
  LIMIT 10
")

tmp

UserId,DisplayName,TotalQuestions,TotalAnswers,TotalComments,TotalScore,AcceptedAnswers
<int>,<chr>,<int>,<int>,<int>,<int>,<int>
25959,Nathan Knutson,15,1556,1385,10368110,632
19705,Criggie,58,3117,14173,174016094,563
24228,Argenti Apparatus,3,1685,2221,15375983,452
8219,Batman,7,873,3432,14222208,321
3924,mattnz,9,851,3045,14186655,250
7309,Chris H,90,930,6552,36599472,203
1584,Daniel R Hicks,16,1025,11828,59305592,198
1259,zenbike,10,465,1187,3160981,196
38270,Weiwen Ng,4,667,2053,5937276,150


## 2.2 Using standard R functions

#### Total comments per user

In [22]:
comments_by_user <- aggregate(
  Id ~ UserId,  # Group by UserId and count Id
  data = Comments,
  FUN = length
)
names(comments_by_user) <- c("UserId", "TotalComments")


#### Total questions and answers per user

In [23]:
questions_answers_by_user <- aggregate(
  Id ~ OwnerUserId + PostTypeId,  # Group by UserId and PostTypeId and count Id
  data = Posts[Posts$PostTypeId %in% c(1, 2), ],
  FUN = length
)

# Pivot PostTypeId into separate columns for questions and answers
questions_answers_wide <- reshape(
  questions_answers_by_user[, c("OwnerUserId", "PostTypeId", "Id")],
  idvar = "OwnerUserId",
  timevar = "PostTypeId",
  direction = "wide"
)

# Rename columns to be more descriptive
names(questions_answers_wide) <- c("UserId", "TotalQuestions", "TotalAnswers")

# Replace NA values with 0
questions_answers_wide$TotalQuestions[
  is.na(questions_answers_wide$TotalQuestions)
] <- 0
questions_answers_wide$TotalAnswers[
  is.na(questions_answers_wide$TotalAnswers)
] <- 0

questions_answers_by_user <- questions_answers_wide


#### Score per user

In [24]:
score_by_user <- aggregate(
  Score ~ OwnerUserId,
  data = Posts,
  FUN = sum
)
names(score_by_user) <- c("UserId", "TotalScore")


#### Accepted answers per user

In [25]:
# Get accepted answers by user
accepted_answers_by_user <- Posts[
  Posts$Id %in% Posts$AcceptedAnswerId[!is.na(Posts$AcceptedAnswerId)],
  c("OwnerUserId", "Id")
]

# Count accepted answers per user
accepted_answers_count <- aggregate(
  Id ~ OwnerUserId,
  data = accepted_answers_by_user,
  FUN = length
)

# Rename columns
names(accepted_answers_count) <- c("UserId", "AcceptedAnswers")


#### Final dataframe

In [26]:
final <- merge(
  accepted_answers_count,
  Users[, c("Id", "DisplayName")],
  by.x = "UserId",
  by.y = "Id"
)

final <- merge(
  final,
  score_by_user,
  by = "UserId"
)

final <- merge(
  final,
  comments_by_user,
  by = "UserId"
)

final <- merge(
  final,
  questions_answers_by_user,
  by = "UserId"
)

# Reorder columns
final <- final[, c(
  "UserId", "DisplayName", "TotalQuestions", "TotalAnswers",
  "TotalComments", "TotalScore", "AcceptedAnswers"
)]

# Sort by AcceptedAnswers descending
final <- final[order(final$AcceptedAnswers, decreasing = TRUE), ]

# Reset row numbers
rownames(final) <- NULL

# Show only top 10
final <- head(final, n = 10)

final


,UserId,DisplayName,TotalQuestions,TotalAnswers,TotalComments,TotalScore,AcceptedAnswers
,<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>
1,25959,Nathan Knutson,15,1556,1385,7486,632
2,19705,Criggie,58,3117,14173,12278,563
3,24228,Argenti Apparatus,3,1685,2221,6923,452
4,8219,Batman,7,873,3432,4144,321
5,3924,mattnz,9,851,3045,4659,250
6,7309,Chris H,90,930,6552,5586,203
7,1584,Daniel R Hicks,16,1025,11828,5014,198
8,1259,zenbike,10,465,1187,2663,196
9,38270,Weiwen Ng,4,667,2053,2892,150


## 2.3 Using dplyr package

In [27]:
# Load required package
library(dplyr)

# Total comments per user
comments_by_user <- Comments %>%
  group_by(UserId) %>%
  summarise(TotalComments = n())

# Total questions and answers per user
questions_answers_by_user <- Posts %>%
  filter(PostTypeId %in% c(1, 2)) %>%
  group_by(OwnerUserId) %>%
  summarise(
    TotalQuestions = sum(PostTypeId == 1),
    TotalAnswers = sum(PostTypeId == 2)
  )

# Score per user
score_by_user <- Posts %>%
  group_by(OwnerUserId) %>%
  summarise(TotalScore = sum(Score))

# Accepted answers per user
accepted_answers_count <- Posts %>%
  filter(Id %in% na.omit(Posts$AcceptedAnswerId)) %>%
  group_by(OwnerUserId) %>%
  summarise(AcceptedAnswers = n()) %>%
  arrange(desc(AcceptedAnswers)) %>%
  head(10)

# Final combined dataframe
final <- accepted_answers_count %>%
  left_join(Users %>% select(Id, DisplayName),
            by = c("OwnerUserId" = "Id")) %>%
  left_join(score_by_user,
            by = c("OwnerUserId" = "OwnerUserId")) %>%
  left_join(comments_by_user,
            by = c("OwnerUserId" = "UserId")) %>%
  left_join(questions_answers_by_user,
            by = c("OwnerUserId" = "OwnerUserId")) %>%
  select(
    UserId = OwnerUserId,
    DisplayName,
    TotalQuestions,
    TotalAnswers,
    TotalComments,
    TotalScore,
    AcceptedAnswers
  ) %>%
  arrange(desc(AcceptedAnswers))

final


UserId,DisplayName,TotalQuestions,TotalAnswers,TotalComments,TotalScore,AcceptedAnswers
<int>,<chr>,<int>,<int>,<int>,<int>,<int>
25959,Nathan Knutson,15,1556,1385,7486,632
19705,Criggie,58,3117,14173,12278,563
24228,Argenti Apparatus,3,1685,2221,6923,452
8219,Batman,7,873,3432,4144,321
3924,mattnz,9,851,3045,4659,250
7309,Chris H,90,930,6552,5586,203
1584,Daniel R Hicks,16,1025,11828,5014,198
1259,zenbike,10,465,1187,2663,196
38270,Weiwen Ng,4,667,2053,2892,150


## 2.4. Comparison and thoughts

#### Benchmarking

In [28]:
res <- microbenchmark(
  base = {
    comments_by_user <- aggregate(
      Id ~ UserId,  # Group by UserId and count Id
      data = Comments,
      FUN = length
    )
    names(comments_by_user) <- c("UserId", "TotalComments")
    questions_answers_by_user <- aggregate(
      Id ~ OwnerUserId + PostTypeId,
      data = Posts[Posts$PostTypeId %in% c(1, 2), ],
      FUN = length
    )
    questions_answers_wide <- reshape(
      questions_answers_by_user[, c("OwnerUserId", "PostTypeId", "Id")],
      idvar = "OwnerUserId",
      timevar = "PostTypeId",
      direction = "wide"
    )
    names(questions_answers_wide) <- c(
      "UserId", "TotalQuestions", "TotalAnswers"
    )
    questions_answers_wide$TotalQuestions[
      is.na(questions_answers_wide$TotalQuestions)
    ] <- 0
    questions_answers_wide$TotalAnswers[
      is.na(questions_answers_wide$TotalAnswers)
    ] <- 0
    questions_answers_by_user <- questions_answers_wide
    score_by_user <- aggregate(
      Score ~ OwnerUserId,
      data = Posts,
      FUN = sum
    )
    names(score_by_user) <- c("UserId", "TotalScore")
    accepted_answers_by_user <- Posts[
      Posts$Id %in% Posts$AcceptedAnswerId[!is.na(Posts$AcceptedAnswerId)],
      c("OwnerUserId", "Id")
    ]
    accepted_answers_count <- aggregate(
      Id ~ OwnerUserId,
      data = accepted_answers_by_user,
      FUN = length
    )
    names(accepted_answers_count) <- c("UserId", "AcceptedAnswers")
    final <- merge(
      accepted_answers_count,
      Users[, c("Id", "DisplayName")],
      by.x = "UserId",
      by.y = "Id"
    )
    final <- merge(final, score_by_user, by = "UserId")
    final <- merge(final, comments_by_user, by = "UserId")
    final <- merge(final, questions_answers_by_user, by = "UserId")
    final <- final[, c(
      "UserId", "DisplayName", "TotalQuestions", "TotalAnswers",
      "TotalComments", "TotalScore", "AcceptedAnswers"
    )]
    final <- final[order(final$AcceptedAnswers, decreasing = TRUE), ]
    rownames(final) <- NULL
    final <- head(final, n = 10)
  },
  dplyr = {
    comments_by_user <- Comments %>%
      group_by(UserId) %>%
      summarise(TotalComments = n())
    questions_answers_by_user <- Posts %>%
      filter(PostTypeId %in% c(1, 2)) %>%
      group_by(OwnerUserId) %>%
      summarise(
        TotalQuestions = sum(PostTypeId == 1),
        TotalAnswers = sum(PostTypeId == 2)
      )
    score_by_user <- Posts %>%
      group_by(OwnerUserId) %>%
      summarise(TotalScore = sum(Score))
    accepted_answers_count <- Posts %>%
      filter(Id %in% na.omit(Posts$AcceptedAnswerId)) %>%
      group_by(OwnerUserId) %>%
      summarise(AcceptedAnswers = n()) %>%
      arrange(desc(AcceptedAnswers)) %>%
      head(10)
    final <- accepted_answers_count %>%
      left_join(Users %>% select(Id, DisplayName),
                by = c("OwnerUserId" = "Id")) %>%
      left_join(score_by_user,
                by = c("OwnerUserId" = "OwnerUserId")) %>%
      left_join(comments_by_user,
                by = c("OwnerUserId" = "UserId")) %>%
      left_join(questions_answers_by_user,
                by = c("OwnerUserId" = "OwnerUserId")) %>%
      select(
        UserId = OwnerUserId,
        DisplayName,
        TotalQuestions,
        TotalAnswers,
        TotalComments,
        TotalScore,
        AcceptedAnswers
      ) %>%
      arrange(desc(AcceptedAnswers))
  },
  sqldf = {
    x <- sqldf("
      SELECT 
        u.Id as UserId,
        u.DisplayName,
        COUNT(
            DISTINCT CASE WHEN p.PostTypeId = 1 THEN p.Id END
        ) as TotalQuestions,
        COUNT(
            DISTINCT CASE WHEN p.PostTypeId = 2 THEN p.Id END
        ) as TotalAnswers,
        COUNT(DISTINCT c.Id) as TotalComments,
        SUM(p.Score) as TotalScore,
        COUNT(
            DISTINCT CASE WHEN p2.AcceptedAnswerId = p.Id THEN p.Id END
        ) as AcceptedAnswers
      FROM Users u
      LEFT JOIN Posts p ON u.Id = p.OwnerUserId
      LEFT JOIN Comments c ON u.Id = c.UserId
      LEFT JOIN Posts p2 ON p2.AcceptedAnswerId = p.Id
      GROUP BY u.Id, u.DisplayName
      HAVING AcceptedAnswers > 0
      ORDER BY AcceptedAnswers DESC
      LIMIT 10
    ")
  }, times = 20
)


#### Result and thoughts

In [29]:
summary(res)

expr,min,lq,mean,median,uq,max,neval
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
base,222.5667,237.6976,245.2374,241.1072,256.5562,272.2792,20
dplyr,101.7753,127.8812,134.2086,132.6005,136.3210,207.5562,20
sqldf,67070.6543,67470.7029,69506.8089,68856.5414,70384.6360,78194.6955,20


It turns out that the dplyr package is the fastest, followed by the base R functions and sqldf, where sqldf is extremely slow.

Using the base R functions is more like using pandas or pyspark in Python. It is more verbose and less readable, but flexible and convienient for ones experienced with python.

Using dplyr or sqldf is more like using SQL. It is more readable and concise, but less flexible and convienient for ones experienced with python.

All three methods returned exactly the same result.


# 3. Task 2

## 3.1 Using sqldf (for reference)

In [97]:
final <- sqldf("
  WITH user_badges AS (
    SELECT DISTINCT u.Id as UserId, u.Location, b.Name as BadgeName
    FROM Users u
    JOIN Badges b ON u.Id = b.UserId
    WHERE b.Name IN ('Teacher', 'Explainer', 'Commentator')
  ),
  post_counts AS (
    SELECT ub.Location,
           COUNT(p.Id) as TotalPosts
    FROM user_badges ub
    LEFT JOIN Posts p ON ub.UserId = p.OwnerUserId
    GROUP BY ub.Location
  ),
  badge_counts AS (
    SELECT ub.Location,
           COUNT(ub.BadgeName) as TotalBadges
    FROM user_badges ub
    GROUP BY ub.Location
  )
  SELECT pc.Location, pc.TotalPosts, TotalBadges
  FROM post_counts pc
  JOIN badge_counts bc ON pc.Location = bc.Location
  WHERE pc.Location IS NOT NULL 
  AND pc.Location <> ''
  AND pc.TotalPosts > 0
  ORDER BY pc.TotalPosts DESC, bc.TotalBadges DESC
")

head(final)


,Location,TotalPosts,TotalBadges
,<chr>,<int>,<int>
1,New Zealand,9975,13
2,Washington DC,5102,5
3,"Seattle, WA, USA",4763,5
4,UK,4643,43
5,"Minnesota, USA",3123,3
6,United States,2883,65


## 3.2 Using standard R functions


#### Users with relevant badges

In [31]:
users_with_badges <- merge(
  Users[, c("Id", "Location")],
  Badges[, c("UserId", "Name")],
  by.x = "Id",
  by.y = "UserId"
)

users_with_badges <- users_with_badges[
  users_with_badges$Name %in% c("Teacher", "Explainer", "Commentator"),
]

names(users_with_badges) <- c("UserId", "Location", "BadgeName")


#### Posts by location


In [32]:
posts_by_location <- merge(
  users_with_badges,
  Posts[, c("OwnerUserId", "Id")],
  by.x = "UserId",
  by.y = "OwnerUserId"
)

posts_by_location <- aggregate(
  Id ~ Location,
  data = posts_by_location,
  FUN = length
)

names(posts_by_location) <- c("Location", "TotalPosts")


#### Badges by location

In [33]:
badges_by_location <- aggregate(
  BadgeName ~ Location,
  data = users_with_badges,
  FUN = length
)

names(badges_by_location) <- c("Location", "TotalBadges")


#### Final dataframe

In [36]:
final <- merge(
  posts_by_location,
  badges_by_location,
  by = "Location"
)

final <- final[final$Location != "" & !is.na(final$Location), ]

final <- final[order(final$TotalPosts, final$TotalBadges, decreasing = TRUE), ]

# reset index
rownames(final) <- NULL

head(final)


,Location,TotalPosts,TotalBadges
,<chr>,<int>,<int>
1,New Zealand,9975,13
2,Washington DC,5102,5
3,"Seattle, WA, USA",4763,5
4,UK,4643,43
5,"Minnesota, USA",3123,3
6,United States,2883,65


## 3.3 Using dplyr package

In [123]:

users_with_badges <- Users %>%
  inner_join(Badges, by = c("Id" = "UserId")) %>%
  filter(Name %in% c("Teacher", "Explainer", "Commentator"))

posts_by_location <- users_with_badges %>%
  inner_join(
    Posts,
    by = c("Id" = "OwnerUserId"),
    relationship = "many-to-many"
  ) %>%
  group_by(Location) %>%
  summarise(TotalPosts = n())

badges_by_location <- users_with_badges %>%
  group_by(Location) %>%
  summarise(TotalBadges = n())

final <- posts_by_location %>%
  left_join(badges_by_location, by = "Location") %>%
  filter(!is.na(Location) & Location != "") %>%
  arrange(desc(TotalPosts), desc(TotalBadges))

head(final)


Location,TotalPosts,TotalBadges
<chr>,<int>,<int>
New Zealand,9975,13
Washington DC,5102,5
"Seattle, WA, USA",4763,5
UK,4643,43
"Minnesota, USA",3123,3
United States,2883,65


## 3.4 Comparison and thoughts


#### Benchmarking

In [37]:
res <- microbenchmark(
  base = {
    users_with_badges <- merge(
      Users[, c("Id", "Location")],
      Badges[, c("UserId", "Name")],
      by.x = "Id",
      by.y = "UserId"
    )
    users_with_badges <- users_with_badges[
      users_with_badges$Name %in% c("Teacher", "Explainer", "Commentator"),
    ]
    names(users_with_badges) <- c("UserId", "Location", "BadgeName")
    posts_by_location <- merge(
      users_with_badges,
      Posts[, c("OwnerUserId", "Id")],
      by.x = "UserId",
      by.y = "OwnerUserId"
    )
    posts_by_location <- aggregate(
      Id ~ Location,
      data = posts_by_location,
      FUN = length
    )
    names(posts_by_location) <- c("Location", "TotalPosts")
    badges_by_location <- aggregate(
      BadgeName ~ Location,
      data = users_with_badges,
      FUN = length
    )
    names(badges_by_location) <- c("Location", "TotalBadges")
    final <- merge(
      posts_by_location,
      badges_by_location,
      by = "Location"
    )
    final <- final[final$Location != "" & !is.na(final$Location), ]
    final <- final[
      order(final$TotalPosts, final$TotalBadges, decreasing = TRUE),
    ]
  },
  dplyr = {
    users_with_badges <- Users %>%
      inner_join(Badges, by = c("Id" = "UserId")) %>%
      filter(Name %in% c("Teacher", "Explainer", "Commentator"))

    posts_by_location <- users_with_badges %>%
      inner_join(
        Posts,
        by = c("Id" = "OwnerUserId"),
        relationship = "many-to-many"
      ) %>%
      group_by(Location) %>%
      summarise(TotalPosts = n())

    badges_by_location <- users_with_badges %>%
      group_by(Location) %>%
      summarise(TotalBadges = n())

    final <- posts_by_location %>%
      left_join(badges_by_location, by = "Location") %>%
      filter(!is.na(Location) & Location != "") %>%
      arrange(desc(TotalPosts), desc(TotalBadges))
  },
  sqldf = {
    sqldf("
      WITH user_badges AS (
        SELECT DISTINCT u.Id as UserId, u.Location, b.Name as BadgeName
        FROM Users u
        JOIN Badges b ON u.Id = b.UserId
        WHERE b.Name IN ('Teacher', 'Explainer', 'Commentator')
      ),
      post_counts AS (
        SELECT ub.Location,
              COUNT(p.Id) as TotalPosts
        FROM user_badges ub
        LEFT JOIN Posts p ON ub.UserId = p.OwnerUserId
        GROUP BY ub.Location
      ),
      badge_counts AS (
        SELECT ub.Location,
              COUNT(ub.BadgeName) as TotalBadges
        FROM user_badges ub
        GROUP BY ub.Location
      )
      SELECT pc.Location, pc.TotalPosts, TotalBadges
      FROM post_counts pc
      JOIN badge_counts bc ON pc.Location = bc.Location
      WHERE pc.Location IS NOT NULL 
      AND pc.Location <> ''
      AND pc.TotalPosts > 0
      ORDER BY pc.TotalPosts DESC, bc.TotalBadges DESC
    ")
  }, times = 20
)


#### Result and thoughts

In [38]:
summary(res)

expr,min,lq,mean,median,uq,max,neval
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
base,225.52243,258.02905,295.29982,276.1190,319.69320,401.5638,20
dplyr,48.68045,52.50622,83.61369,85.2052,95.25614,225.8468,20
sqldf,389.21960,398.95921,412.51305,404.9089,416.35900,471.1348,20


Again, the dplyr package is the fastest, followed by the base R functions and sqldf.

All three methods returned exactly the same result.


# 4. Task 3

## 4.1 Using sqldf (for reference)


In [39]:
res <- sqldf("
  SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
  FROM (
    SELECT
      PostId,
      COUNT(*) AS Count,
      STRFTIME('%Y', Votes.CreationDate) AS Year
    FROM Votes
    WHERE VoteTypeId=2
    GROUP BY PostId, Year
  ) AS UpVotesPerYear
  JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
  WHERE Posts.PostTypeId=1
  GROUP BY Year
  ORDER BY Year ASC
")

head(res)


,Title,Year,Count
,<chr>,<chr>,<int>
1,Why ride a fixed-gear bike?,2010,21
2,Why ride a fixed-gear bike?,2011,31
3,How do I deal with a dog chasing me when I'm touring?,2012,41
4,Ride with someone who is less trained,2013,34
5,How to get over anger at inconsiderate drivers,2014,33
6,How to commute to work on your bike and dress up,2015,39


## 4.2 Using standard R functions

I choose to use standard R functions for this task as I find it personally more convenient and readable.


In [62]:
upvotes_per_year <- aggregate(
  VoteTypeId ~ PostId + strftime(CreationDate, "%Y"),
  data = Votes[Votes$VoteTypeId == 2, ],
  FUN = length
)
names(upvotes_per_year) <- c("PostId", "Year", "Count")

questions <- Posts[Posts$PostTypeId == 1, ]

final <- merge(
  upvotes_per_year,
  questions[, c("Id", "Title")],
  by.x = "PostId",
  by.y = "Id"
)

# Get max count per year
max_counts <- aggregate(
  Count ~ Year,
  data = final,
  FUN = max
)

# Join back with original data to get titles
final <- merge(
  final,
  max_counts,
  by = c("Year", "Count")
)

final <- final[, c("Title", "Year", "Count")]

final <- final[order(final$Year), ]

rownames(final) <- NULL

head(final)


,Title,Year,Count
,<chr>,<chr>,<int>
1,Why ride a fixed-gear bike?,2010,21
2,Why ride a fixed-gear bike?,2011,31
3,How do I deal with a dog chasing me when I'm touring?,2012,41
4,Ride with someone who is less trained,2013,34
5,How to get over anger at inconsiderate drivers,2014,33
6,How to commute to work on your bike and dress up,2015,39


## 4.3 Explanation of the query



The query returns the most upvoted question post for each year, showing:

- Title: The title of the question post
- Year: The year the upvotes were received 
- Count: The number of upvotes received in that year

It does this by:

1. First getting upvote counts per post per year from the Votes table
2. Joining with Posts to get the post titles
3. Filtering for only question posts (PostTypeId=1)
4. Finding the post with the maximum upvotes for each year
5. Ordering chronologically by year
